# Step 1: Import required libraries

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from itertools import product

# Step 2: Data Handling

In [2]:
#Path to Excel file
file_path = 'Updated_Project_Requirements.xlsx'
file_path1 = 'Subset_Student_responses.xlsx'

# Read the Excel file into a DataFrame
project_requirements_df = pd.read_excel(file_path)
student_responses_df = pd.read_excel(file_path1)


In [3]:
project = project_requirements_df['Project Name']
project_social_impact = project_requirements_df['Social Impact']
project_technology_stratergy = project_requirements_df['Technology Strategy']
project_market_development = project_requirements_df['Product/ Market Development']
project_business_model = project_requirements_df['Business Model/Process Transformation']
project_data_science = project_requirements_df['Data Science/ Business Analytics']
project_capacity = project_requirements_df['Project Capacity']

students = student_responses_df['Response ID']
first_choice = student_responses_df['Top 3 projects - First Choice']
second_choice = student_responses_df['Top 3 projects - Second Choice']
third_choice = student_responses_df['Top 3 projects - Third Choice']
students_social_impact = student_responses_df['Preferences for project categories - Social Impact\n\nNote that rank 1 indicates your highest interest and 5 indicates your lowest interest. - Social Impact']
students_technology_stratergy = student_responses_df['Preferences for project categories - Technology Strategy\n\nNote that rank 1 indicates your highest interest and 5 indicates your lowest interest. - Technology Strategy']
students_market_development = student_responses_df['Preferences for project categories - Product/Market Development\n\nNote that rank 1 indicates your highest interest and 5 indicates your lowest interest. - Product/Market Development'] 
students_business_model = student_responses_df['Preferences for project categories - Business Model/Process\n\nNote that rank 1 indicates your highest interest and 5 indicates your lowest interest. - Business Model/Process']
students_data_science = student_responses_df['Preferences for project categories - Data Science/Business Analytics\n\nNote that rank 1 indicates your highest interest and 5 indicates your lowest interest. - Data Science/Business Analytics']


# Step 3: Mapping and Scoring 

In [4]:
# Categories and preferences
categories = [
    'Social Impact', 'Technology Strategy', 'Product/ Market Development',
    'Business Model/Process Transformation', 'Data Science/ Business Analytics'
]

# Mapping from DataFrame columns to category names
project_category_map = {
    'Social Impact': project_social_impact,
    'Technology Strategy': project_technology_stratergy,
    'Product/ Market Development': project_market_development,
    'Business Model/Process Transformation': project_business_model,
    'Data Science/ Business Analytics': project_data_science
}

student_preference_map = {
    'Social Impact': students_social_impact,
    'Technology Strategy': students_technology_stratergy,
    'Product/ Market Development': students_market_development,
    'Business Model/Process Transformation': students_business_model,
    'Data Science/ Business Analytics': students_data_science
}
# Inverting the student preference scores
for category in categories:
    student_preference_map[category] = 6 - student_preference_map[category]

# Initialize a DataFrame to store compatibility scores
compatibility_scores = pd.DataFrame(0, index=student_responses_df.index, columns=project_requirements_df.index)

# Modify the weights for choices
top_choice_weight = 100  # Significantly high score for top 3 choices
category_max_score = 5   # Maximum score a project can get based on category preferences

# Calculate compatibility scores
for student, project_idx in product(compatibility_scores.index, compatibility_scores.columns):
    project_name = project_requirements_df.iloc[project_idx]['Project Name']
    # Sum of category scores, normalized
    category_score = sum(
        project_category_map[cat][project_idx] * student_preference_map[cat][student]
        for cat in categories
    ) / len(categories) * category_max_score

    # Explicitly high scores for top three choices
    choice_score = 0
    if project_name == first_choice[student]:
        choice_score = top_choice_weight
    elif project_name == second_choice[student]:
        choice_score = top_choice_weight * 0.8  # Slightly lower for second choice
    elif project_name == third_choice[student]:
        choice_score = top_choice_weight * 0.6  # Even lower for third choice

    # Calculate final compatibility score
    compatibility_scores.at[student, project_idx] = category_score + choice_score


# Step 4: Create an Optimization Model

In [5]:
# Create a Gurobi model
m = gp.Model("project_assignment")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-21


# Step 5: Add variables

In [6]:
# Replace NaN or infinite values in compatibility_scores with 0
compatibility_scores = compatibility_scores.replace([np.inf, -np.inf], np.nan)
compatibility_scores.fillna(0, inplace=True)


# Before setting the objective, check for NaN or Inf values
if np.any(np.isnan(compatibility_scores)) or np.any(np.isinf(compatibility_scores)):
    raise ValueError("compatibility_scores contains NaN or infinite values")

# Decision variables for student-project assignments
assignments = m.addVars(
    compatibility_scores.index, compatibility_scores.columns,
    vtype=GRB.BINARY, name="assign"
)


# Step 6: Add Constraints

In [7]:
# Add constraints for each student to be assigned to exactly one project
for s in compatibility_scores.index:
    m.addConstr(
        gp.quicksum(assignments[s, p] for p in compatibility_scores.columns) == 1,
        name=f"assign_one_project_{s}"
    )

# Add constraints for project capacities
# Troubleshooting: Verify that project_capacity matches the projects in compatibility_scores.columns
for p in compatibility_scores.columns:
    project_capacity_value = project_capacity[p]  # Check the capacity for this project
    assigned_students_sum = gp.quicksum(assignments[s, p] for s in compatibility_scores.index)
    m.addConstr(
        assigned_students_sum <= project_capacity_value,
        name=f"project_capacity_{p}"
    )

# Step 7: First Priority: Optimize for Compatibility Score

In [8]:
# Primary Objective: Maximize the sum of compatibility scores
primary_objective = gp.quicksum(
    assignments[s, p] * compatibility_scores.at[s, p]
    for s in compatibility_scores.index
    for p in compatibility_scores.columns
)
# Set the objective to be the primary objective
m.setObjective(primary_objective, GRB.MAXIMIZE)
# Optimize the model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 254 rows, 8385 columns and 16770 nonzeros
Model fingerprint: 0x0383a417
Variable types: 0 continuous, 8385 integer (8385 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 173030.00000
Presolve time: 0.01s
Presolved: 254 rows, 8385 columns, 16770 nonzeros
Variable types: 0 continuous, 8385 integer (8385 binary)
Found heuristic solution: objective 189530.00000

Root relaxation: objective 2.136900e+05, 783 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    213690.00000 213690.000  0

# Step 8: Add the solution from the first priority as a constraint & 
   # Second Priority: Optimize for Top 3 Choices

In [9]:
# Secondary Objective: Maximize assignments from top three choices
secondary_objective = gp.quicksum(
    assignments[s, p] * top_choice_weight
    for s in compatibility_scores.index
    for p in compatibility_scores.columns
    if project_name in [first_choice[s], second_choice[s], third_choice[s]]
)
# Add the solution from the first priority as a constraint
m.addConstr(primary_objective <= m.getObjective().getValue())
# Set the objective to be the secondary objective
m.setObjective(secondary_objective, GRB.MAXIMIZE)
# Optimize the model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 255 rows, 8385 columns and 24882 nonzeros
Model fingerprint: 0x94cfbb0e
Variable types: 0 continuous, 8385 integer (8385 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

Loaded MIP start from previous solve with objective -0


Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


# Step 9: Output the results

In [10]:
# Extract the assignments from the model
assignment_results = []

if m.status == GRB.OPTIMAL:
    for s in compatibility_scores.index:
        for p in compatibility_scores.columns:
            if assignments[s, p].X > 0.5:  # If the student-project pair is selected
                # Use the project title mapping to get the descriptive title
                assignment_results.append({
                    'Student': students.iloc[s],  # Assuming 's' is the student ID (Response ID)
                    'Project': project.iloc[p], 
                    'Assigned': assignments[s, p].X # Use the descriptive title
                })
# Convert the results to a DataFrame
assignment_results_df = pd.DataFrame(assignment_results)

# Define the file path for the CSV file
csv_file_path = 'Result.csv'

# Write the DataFrame to a CSV file
assignment_results_df.to_csv(csv_file_path, index=False)

# Output the file path
print(f"Assignments have been written to {csv_file_path}")

Assignments have been written to Result.csv


In [11]:
# Output the file path
for p in compatibility_scores.columns:
    project_capacity_value = project_capacity[p]
    assigned_students_sum = sum(assignments[s, p].X for s in compatibility_scores.index)
    print(f"Project: {p}, Capacity: {project_capacity_value}, Assigned: {assigned_students_sum}")

Project: 0, Capacity: 5, Assigned: 5.0
Project: 1, Capacity: 5, Assigned: 5.0
Project: 2, Capacity: 7, Assigned: 7.0
Project: 3, Capacity: 5, Assigned: 5.0
Project: 4, Capacity: 6, Assigned: 6.0
Project: 5, Capacity: 5, Assigned: 5.0
Project: 6, Capacity: 6, Assigned: 6.0
Project: 7, Capacity: 5, Assigned: 5.0
Project: 8, Capacity: 5, Assigned: 5.0
Project: 9, Capacity: 5, Assigned: 5.0
Project: 10, Capacity: 6, Assigned: 6.0
Project: 11, Capacity: 8, Assigned: 6.0
Project: 12, Capacity: 8, Assigned: 8.0
Project: 13, Capacity: 8, Assigned: 8.0
Project: 14, Capacity: 15, Assigned: 15.0
Project: 15, Capacity: 5, Assigned: 2.0
Project: 16, Capacity: 5, Assigned: 5.0
Project: 17, Capacity: 5, Assigned: 2.0
Project: 18, Capacity: 6, Assigned: 5.0
Project: 19, Capacity: 5, Assigned: 5.0
Project: 20, Capacity: 10, Assigned: 10.0
Project: 21, Capacity: 10, Assigned: 10.0
Project: 22, Capacity: 10, Assigned: 10.0
Project: 23, Capacity: 4, Assigned: 4.0
Project: 24, Capacity: 5, Assigned: 5.0
Pr